In [1]:
import argparse
import torch
from torchvision import datasets, transforms, models
import numpy as np
import time
import copy
import PIL
import json
import argparse
import logging
import os
import sys
import utility
from collections import OrderedDict

logging.basicConfig(stream=sys.stdout, level=logging.INFO)


data_transforms = utility.data_transforms

In [42]:
def load_model(path, device='cpu'):
    logging.info(f"Loading {path}")
    #checkpoint = torch.load(path,map_location=device)
    checkpoint = torch.load(path)

    architecture = checkpoint['arch']
    pretrained_model = getattr(models, architecture)

    model = pretrained_model(pretrained = True)
    in_features = checkpoint['in_features']
    num_classes = len(checkpoint['class_to_idx'])
    
    hidden_units = checkpoint['hidden_units']
    
    classifier_layer = checkpoint['classifier_layer']
    
    print (checkpoint.keys())
    logging.info(classifier_layer)
    additional_layers  =  torch.nn.Sequential(OrderedDict([
                          ('fc1', torch.nn.Linear(in_features, hidden_units)),
                          ('relu', torch.nn.ReLU()),
                          ('fc2', torch.nn.Linear(hidden_units, num_classes)),
                          ('output', torch.nn.LogSoftmax(dim=1))
                          ]))

    
    return model, classifier_layer, additional_layers
    setattr(model, classifier_layer,classifier_layer)


    model.load_state_dict(checkpoint['state'])
    model.class_to_idx = checkpoint['class_to_idx']


    return model

In [43]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    img = PIL.Image.open(image)
    img_tensor = data_transforms["test"](img)
    
    num_dim = img_tensor.shape[0]
    if num_dim > 3:
        img_tensor = img_tensor[:3,::]
    
    return img_tensor.numpy()

In [44]:
category_names='cat_to_name.json'
checkpoint='checkpoints/checkpoint.pth'
image_path ='assets/inference_example.png'

In [45]:
with open(category_names, 'r') as f:
    cat_to_name = json.load(f)    

#model = load_model(checkpoint, device='cpu')

model, classifier_layer, additional_layers = load_model(checkpoint, device='cpu')

INFO:root:Loading checkpoints/checkpoint.pth
dict_keys(['arch', 'state', 'class_to_idx', 'in_features', 'hidden_units', 'classifier_layer'])
INFO:root:fc


In [48]:
additional_layers

Sequential(
  (fc1): Linear(in_features=2048, out_features=512, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=512, out_features=102, bias=True)
  (output): LogSoftmax()
)

In [47]:
classifier_layer

'fc'

In [46]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [52]:
model.fc = 

Linear(in_features=2048, out_features=1000, bias=True)

In [53]:
setattr(model, 'fc',additional_layers)

In [49]:
setattr(model, classifier_layer,classifier_layer)

TypeError: cannot assign 'str' as child module 'fc' (torch.nn.Module or None expected)

In [37]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [23]:
model.eval()
img = process_image(image_path)
img_tensor = torch.from_numpy(img).unsqueeze(0).type(torch.FloatTensor)

# feed image to model
logits = model(img_tensor)


In [24]:
logits

tensor([[ -4.1061,  -4.6797,  -8.2874,  -6.3549,  -9.3021,  -7.8603,
          -8.4437,  -7.3188,  -6.2680,  -5.3549,  -6.4298,  -5.6309,
          -6.5109,  -2.5765,  -6.3563,  -4.3419,  -6.3059,  -3.0034,
          -6.3683,  -7.3418,  -3.6886,  -5.8682,  -6.1253,  -2.5794,
          -5.3186,  -3.7060,  -5.0265,  -5.7640,  -4.9525,  -3.5274,
          -3.8095,  -4.3672,  -6.4158,  -6.2784,  -5.0107,  -8.6609,
          -4.4745,  -6.6745,  -8.1653,  -5.6574,  -5.7794,  -5.9470,
          -3.3539,  -5.9712,  -8.2896,  -7.6132,  -7.8546,  -7.4096,
          -7.0612,  -5.6606,  -7.0944,  -6.0501,  -4.0389,  -3.4038,
          -6.4569,  -4.0905, -10.7095,  -7.7651,  -5.7966, -13.2546,
          -7.0229,  -4.0436,  -5.8071,  -6.4283,  -6.1637,  -5.5662,
          -3.8922,  -5.3840,  -5.1460,  -8.0875,  -4.8407, -10.0810,
          -6.3495,  -9.9712,  -8.9771,  -5.8456,  -2.5403,  -5.5489,
          -8.5011,  -3.1312,  -9.2589,  -8.6674, -10.2983,  -4.7027,
          -4.4393,  -5.0559,  -9.3

In [31]:
probs  = torch.exp(logits)
top_probs, top_labels = probs.topk(5)
top_probs, top_labels = top_probs.data.tolist()[0]  , top_labels.data.tolist()[0]

(top_probs, top_labels)

([0.13085201382637024,
  0.07884562760591507,
  0.07603628933429718,
  0.07581935077905655,
  0.049619778990745544],
 [100, 76, 13, 23, 17])

In [29]:
probs.topk

<function Tensor.topk>

In [ ]:
import numpy as np
exps = [np.exp(i) for i in logits.detach().numpy()]

In [ ]:
l = logits.detach()

In [ ]:
l.data

In [ ]:
exps

In [ ]:
logits /   sum(logits)

In [ ]:
torch.nn.functional.softmax(logits, dim=1)

In [ ]:
torch.exp(torch.nn.Softmax()(logits))

In [ ]:
# get probabilities
probs  = torch.exp(logits)
# get top k
top_probs, top_labels = probs.topk(topk)
top_probs, top_labels = top_probs.data.tolist()[0]  , top_labels.data.tolist()[0]


# Convert indices to classes
idx_to_class = {val: key for key, val in    
                                  model.class_to_idx.items()}

top_labels = [idx_to_class[lab] for lab in top_labels]